# Capstone Project

This notebooke will be mainly used for the capstone project.

In [86]:
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import folium

import numpy as np

from bs4 import BeautifulSoup

import re

## Data wrapping
### Get postal codes of munich

In [63]:
url = 'https://www.dasoertliche.de/Themen/Postleitzahlen/M%C3%BCnchen.html'

df = pd.read_html(url, flavor='bs4')[0]
df

,PLZ,Ortsname,Ortsteil,Landkreis,Bundesland
0,80331,München,AltstadtIsarvorstadtLehelLudwigsvorstadt,Stadt München,Bayern
1,80333,München,AltstadtMaxvorstadtSchwabing-West,Stadt München,Bayern
2,80335,München,AltstadtLudwigsvorstadtMaxvorstadtNeuhausen,Stadt München,Bayern
3,80336,München,AltstadtIsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern
4,80337,München,IsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern
...,...,...,...,...,...
89,82024,Taufkirchen Kr München,NaN,München,Bayern
90,99438,Bad Berka,München Stadt Bad Berka,Weimarer Land,Thüringen
91,92275,Hirschbach Oberpf,München Gem. Hirschbach,Amberg-Sulzbach,Bayern
92,94116,Hutthurm,München Gem. Hutthurm,Passau,Bayern


### Check data types

In [64]:
df.dtypes

PLZ            int64
Ortsname      object
Ortsteil      object
Landkreis     object
Bundesland    object
dtype: object

### Translations of column names

|german|english|
|------|-------|
|PLZ|postal code|
|Ortsname|town name|
|Ortsteil|district|
|Landkreis|borough|
|Bundesland|state|

### Select rows
- Select rows inlcuding *Stadt München* in column Landkreis.
- Select rows excluding *München-Flughafen* in column Ortsname.

In [65]:
df = df.loc[df.Landkreis == 'Stadt München'].reset_index(drop=True)
df = df.loc[~(df.Ortsname == 'München-Flughafen')]
df

,PLZ,Ortsname,Ortsteil,Landkreis,Bundesland
0,80331,München,AltstadtIsarvorstadtLehelLudwigsvorstadt,Stadt München,Bayern
1,80333,München,AltstadtMaxvorstadtSchwabing-West,Stadt München,Bayern
2,80335,München,AltstadtLudwigsvorstadtMaxvorstadtNeuhausen,Stadt München,Bayern
3,80336,München,AltstadtIsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern
4,80337,München,IsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern
...,...,...,...,...,...
69,81827,München,PerlachTrudering,Stadt München,Bayern
70,81829,München,Berg am LaimRiemTrudering,Stadt München,Bayern
71,81925,München,Bogenhausen,Stadt München,Bayern
72,81927,München,Bogenhausen,Stadt München,Bayern


### Create new column *location*:

Bundesland + Ortsname + PLZ

In [66]:
df['location'] = df.Bundesland + ', ' + df.Ortsname + ', ' + df.PLZ.astype(str)
df

,PLZ,Ortsname,Ortsteil,Landkreis,Bundesland,location
0,80331,München,AltstadtIsarvorstadtLehelLudwigsvorstadt,Stadt München,Bayern,"Bayern, München, 80331"
1,80333,München,AltstadtMaxvorstadtSchwabing-West,Stadt München,Bayern,"Bayern, München, 80333"
2,80335,München,AltstadtLudwigsvorstadtMaxvorstadtNeuhausen,Stadt München,Bayern,"Bayern, München, 80335"
3,80336,München,AltstadtIsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern,"Bayern, München, 80336"
4,80337,München,IsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern,"Bayern, München, 80337"
...,...,...,...,...,...,...
69,81827,München,PerlachTrudering,Stadt München,Bayern,"Bayern, München, 81827"
70,81829,München,Berg am LaimRiemTrudering,Stadt München,Bayern,"Bayern, München, 81829"
71,81925,München,Bogenhausen,Stadt München,Bayern,"Bayern, München, 81925"
72,81927,München,Bogenhausen,Stadt München,Bayern,"Bayern, München, 81927"


### Geocode

- Latitude
- Longitude

In [80]:
user_agent = 'coursera_capstone_munich'

In [67]:
gdf = gpd.tools.geocode(df.location, provider='nominatim', user_agent=user_agent)
gdf

,geometry,address
0,POINT (11.57344 48.13606),"Altstadt-Lehel, München, Bayern, 80331, Deutsc..."
1,POINT (11.56923 48.14551),"Maxvorstadt, München, Bayern, 80333, Deutschland"
2,POINT (11.55582 48.14572),"Maxvorstadt, München, Bayern, 80335, Deutschland"
3,POINT (11.55672 48.13410),"Ludwigsvorstadt-Isarvorstadt, München, Bayern,..."
4,POINT (11.55795 48.12722),"Ludwigsvorstadt-Isarvorstadt, München, Bayern,..."
...,...,...
69,POINT (11.69318 48.10950),"Trudering-Riem, München, Bayern, 81827, Deutsc..."
70,POINT (11.67779 48.13200),"Trudering-Riem, München, Bayern, 81829, Deutsc..."
71,POINT (11.62170 48.16174),"Bezirksteil Herzogpark, Bogenhausen, München, ..."
72,POINT (11.63748 48.15747),"Bezirksteil Englschalking, Bogenhausen, Münche..."


### Create columns *latitude* and *longitude*

In [96]:
gdf['latitude'] = gdf.geometry.y
gdf['longitude'] = gdf.geometry.x

### Merge df and gdf

In [97]:
df_merge = df.merge(gdf, left_index=True, right_index=True)
df_merge

,PLZ,Ortsname,Ortsteil,Landkreis,Bundesland,location,geometry,address,latitude,longitude
0,80331,München,AltstadtIsarvorstadtLehelLudwigsvorstadt,Stadt München,Bayern,"Bayern, München, 80331",POINT (11.57344 48.13606),"Altstadt-Lehel, München, Bayern, 80331, Deutsc...",48.136057,11.573444
1,80333,München,AltstadtMaxvorstadtSchwabing-West,Stadt München,Bayern,"Bayern, München, 80333",POINT (11.56923 48.14551),"Maxvorstadt, München, Bayern, 80333, Deutschland",48.145514,11.569229
2,80335,München,AltstadtLudwigsvorstadtMaxvorstadtNeuhausen,Stadt München,Bayern,"Bayern, München, 80335",POINT (11.55582 48.14572),"Maxvorstadt, München, Bayern, 80335, Deutschland",48.145724,11.555819
3,80336,München,AltstadtIsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern,"Bayern, München, 80336",POINT (11.55672 48.13410),"Ludwigsvorstadt-Isarvorstadt, München, Bayern,...",48.134098,11.556723
4,80337,München,IsarvorstadtLudwigsvorstadtSendling,Stadt München,Bayern,"Bayern, München, 80337",POINT (11.55795 48.12722),"Ludwigsvorstadt-Isarvorstadt, München, Bayern,...",48.127223,11.557946
...,...,...,...,...,...,...,...,...,...,...
69,81827,München,PerlachTrudering,Stadt München,Bayern,"Bayern, München, 81827",POINT (11.69318 48.10950),"Trudering-Riem, München, Bayern, 81827, Deutsc...",48.109498,11.693181
70,81829,München,Berg am LaimRiemTrudering,Stadt München,Bayern,"Bayern, München, 81829",POINT (11.67779 48.13200),"Trudering-Riem, München, Bayern, 81829, Deutsc...",48.132005,11.677786
71,81925,München,Bogenhausen,Stadt München,Bayern,"Bayern, München, 81925",POINT (11.62170 48.16174),"Bezirksteil Herzogpark, Bogenhausen, München, ...",48.161737,11.621705
72,81927,München,Bogenhausen,Stadt München,Bayern,"Bayern, München, 81927",POINT (11.63748 48.15747),"Bezirksteil Englschalking, Bogenhausen, Münche...",48.157465,11.637478


## Create map of Munich
### Get coordinates of Munich

In [94]:
munich = gpd.tools.geocode('munich', user_agent=user_agent)

latitude = munich.geometry.y
longitude = munich.geometry.x

munich

,geometry,address
0,POINT (11.58022 48.13913),"Munich, BY, Germany, Germany"


In [99]:
# create map of Munich using latitude and longitude values
map_munich = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_merge['latitude'], df_merge['longitude'], df_merge['address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)  
    
map_munich

## Fousquare API
### Read credentials

In [57]:
import yaml

with open('.credentials.yaml', 'r') as creds:
    credentials = yaml.safe_load(creds)
    
CLIENT_ID = credentials['foursquare']['client_id']
CLIENT_SECRET = credentials['foursquare']['client_secret']
VERSION = '20210320' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value